In [1]:
import os
os.chdir('../..')

In [2]:
from modules.table_evaluator import load_data, TableEvaluator
# From https://github.com/Baukebrenninkmeijer/table-evaluator

from os import listdir
from os.path import isfile, join

import matplotlib.pyplot as plt
import numpy as np

import pandas as pd

import pickle

# For the Python notebook
%matplotlib inline
%reload_ext autoreload
%autoreload 2

# Functions

In [3]:
def is_a_DATGAN(name):
    if any(x in name for x in ['TGAN', 'CTGAN', 'TVAE', 'FULL', 'TRANSRED', 'LINEAR', 'NOLINKS', 'PREDICTION']):
        return False
    else:
        return True
    
def check_low_appearing_vars(df):
    
    for c in df.columns:
        val = df[c].value_counts()
        if len(val) < 20:
            val = val/len(df)
            if any(val < 0.01) and c != 'choice':
                print('Variable {}: '.format(c))
                for idx, v in zip(val.index, val):
                    if v < 0.01:
                        print('  {} - {:.2f}% ({:d})'.format(idx, 100*v, int(v*len(df))))
                print()
                
def replace_low_appearing_values(df, dataset):
    
    if 'Chicago' in dataset:
        dct_ = {}
        for i in df['hh_vehicles'].unique():
            if i >= 5:
                dct_[i] = '5+'
            else:
                dct_[i] = str(i)        
        df['hh_vehicles'].replace(dct_, inplace=True)
        
        dct_ = {}
        for i in df['hh_size'].unique():
            if i >= 6:
                dct_[i] = '6+'
            else:
                dct_[i] = str(i)        
        df['hh_size'].replace(dct_, inplace=True)
        
        dct_ = {}
        for i in df['hh_bikes'].unique():
            if i >= 6:
                dct_[i] = '6+'
            else:
                dct_[i] = str(i)        
        df['hh_bikes'].replace(dct_, inplace=True)       

    elif 'LPMC' in dataset:
        dct_ = {}
        for i in df['pt_n_interchanges'].unique():
            if i >= 2:
                dct_[i] = '2+'
            else:
                dct_[i] = str(i)        
        df['pt_n_interchanges'].replace(dct_, inplace=True) 
        
        dct_ = {
            'Diesel_LGV': 'LGV',
            'Petrol_LGV': 'LGV',
            'Hybrid_Car': 'Average_Car'
        }

In [4]:
dataset = 'Chicago_DAG'
orig_str = 'random-original'
input_folder = '../synth_data/{}/'.format(dataset)
n_models = 5
n_data = 5

# Models for testing all DATGANS
models = ['CTGAN', 'TGAN', 'TVAE']

for i in ['WGAN', 'SGAN', 'WGGP']:
    for j in ['WI', 'OR', 'WO']:
        for k in ['NO', 'BO', 'OD', 'OC']:
            models.append('{}_{}_{}'.format(i,j,k))
            
# Models for testing different DAGs
if 'DAG' in dataset:
    models = ['FULL', 'TRANSRED', 'LINEAR', 'NOLINKS', 'PREDICTION']

models.sort()

files_ = {}

for m in models:
    tmp = []
    if is_a_DATGAN(m):
        spl = m.split('_')
        for i in range(n_models):
            for j in range(n_data):
                tmp.append(input_folder + '{}_{}_{:0>2}_{}_{:0>2}.csv'.format(spl[0], spl[1], i+1,  spl[2], j+1))
    else:
        for i in range(n_models):
            for j in range(n_data):
                tmp.append(input_folder + '{}_{:0>2}_{:0>2}.csv'.format(m, i+1, j+1))
    files_[m] = tmp

In [5]:
if 'Chicago' in dataset:
    continuous_columns = ["distance", "age", "departure_time"]
elif 'LPMC' in dataset:
    continuous_columns = ['start_time_linear', 'age', 'distance', 'dur_walking', 'dur_cycling', 'dur_pt_access', 'dur_pt_rail', 'dur_pt_bus', 'dur_pt_int', 'dur_driving', 'cost_transit', 'cost_driving_fuel', 'driving_traffic_percent']

In [6]:
df_orig = pd.read_csv('../data/' + dataset.split('_')[0] + '/data.csv')

In [7]:
replace_low_appearing_values(df_orig, dataset)

In [8]:
check_low_appearing_vars(df_orig)

In [9]:
cat_cols = list(set(df_orig.columns) - set(continuous_columns))

In [10]:
filepath = './notebooks/results/{}/'.format(dataset)
filename = 'te_results.pickle'
te_results = {}
n = 7000

if 'Chicago' in dataset:
    target_col = 'choice'
elif 'LPMC' in dataset:
    target_col = 'mode_choice'

try:
    te_results = pickle.load(open(f'{filepath}{filename}','rb'))
    print('Found previous picle file, using that')
except:
    print('No previous results found, starting fresh')
    try:
        os.makedirs(filepath)
    except:
        pass

Found previous picle file, using that


In [11]:
for i, model in enumerate(models):
    
    if model in te_results.keys():
        print("Results for model \033[1m{}\033[0m ({}/{}) already exists".format(model, i+1, len(models)))
    else:
        print("Getting results for model \033[1m{}\033[0m ({}/{})".format(model, i+1, len(models)))
        te_results[model] = []
        
    n_files_done = len(te_results[model])
    
    for j, f in enumerate(files_[model][n_files_done:]):
        print("  Processing file {}/{}".format(j+1+n_files_done, len(files_[model])) + " "*20)

        tmp_df = pd.read_csv(f)
        replace_low_appearing_values(tmp_df, dataset)
        
        te = TableEvaluator(tmp_df, df_orig, cat_cols, verbose=False)
        
        res = te.evaluate(target_col=target_col, kfold=True)
        
        te_results[model].append(res.content.to_dict()['result'])
        
        pickle.dump(te_results, open(filepath + filename, 'wb'))

Results for model FULL (1/5) already exists
Results for model LINEAR (2/5) already exists
Results for model NOLINKS (3/5) already exists
Results for model PREDICTION (4/5) already exists
Results for model TRANSRED (5/5) already exists


In [12]:
if orig_str not in te_results:
    te_results[orig_str] = []

    for i in range(n_models*n_data):
        print("Processing random dataset {}/{}".format(i+1, len(files_[model])) + " "*20)

        train = df_orig.sample(int(len(df_orig) * 0.5))
        train.index = range(len(train))
        test = df_orig[~df_orig.index.isin(train.index)]
        test.index = range(len(test))
        
        te = TableEvaluator(test, train, cat_cols, verbose=False)
        
        res = te.evaluate(target_col=target_col, kfold=True)
        
        te_results[orig_str].append(res.content.to_dict()['result'])
        
        pickle.dump(te_results, open(filepath + filename, 'wb'))

In [13]:
keys = te_results[orig_str][0].keys()

In [14]:
res = {}

for k in keys:
    res[k] = {}
            
    for m in te_results.keys():
        
        tmp = []
        
        for i in range(n_models*n_data):
            tmp.append(te_results[m][i][k])
            
        res[k][m] = {
            'mean': np.mean(tmp),
            'std': np.std(tmp)
        }

In [15]:
for s in keys:
    print('Ranking on "{}":'.format(s))

    sorted_dct = {k: v for k, v in sorted(res[s].items(), key=lambda item: item[1]['mean'])[::-1]}

    for i, item in enumerate(sorted_dct):
        print('  {:>2}. {:<15} - {:.2e} ± {:.2e}'.format(i+1, item, sorted_dct[item]['mean'], sorted_dct[item]['std']))
    print()


Ranking on "Basic statistics":
   1. random-original - 1.00e+00 ± 0.00e+00
   2. LINEAR          - 9.97e-01 ± 6.89e-03
   3. FULL            - 9.93e-01 ± 9.06e-03
   4. NOLINKS         - 9.86e-01 ± 9.28e-03
   5. PREDICTION      - 9.85e-01 ± 8.92e-03
   6. TRANSRED        - 9.82e-01 ± 8.42e-03

Ranking on "Correlation column correlations":
   1. random-original - 9.96e-01 ± 0.00e+00
   2. FULL            - 9.85e-01 ± 1.68e-03
   3. TRANSRED        - 9.84e-01 ± 3.17e-03
   4. LINEAR          - 9.77e-01 ± 1.86e-03
   5. NOLINKS         - 4.32e-01 ± 5.20e-02
   6. PREDICTION      - 3.88e-01 ± 3.97e-02

Ranking on "Mean Correlation between fake and real columns":
   1. PREDICTION      - 9.52e-01 ± 3.75e-03
   2. NOLINKS         - 9.50e-01 ± 4.89e-03
   3. TRANSRED        - 9.35e-01 ± 7.42e-03
   4. LINEAR          - 9.33e-01 ± 6.78e-03
   5. FULL            - 9.23e-01 ± 9.96e-03
   6. random-original - 9.17e-01 ± 1.11e-16

Ranking on "1 - MAPE Estimator results":
   1. TRANSRED        - 9.